# Registration Loading

Notebook to load registration data into the database. This notebook demonstrates how to create registration entries with their associated metadata.

## Imports

In [ ]:
import psycopg2
import numpy as np
from tabulate import tabulate
# Import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.load as load

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

## Registration Data

The `load_registration` function creates a new registration entry in the database with associated metadata.

Parameters:
- `transformation_matrix`: The 3x3 transformation matrix used for registration
- `reference_file_path`: The file path of the reference measurement
- `registered_file_path`: The file path of the registered measurement
- `additional_metadata`: Optional list of dictionaries with metadata

Each metadata dictionary should contain:
- `key`: The name of the metadata field
- `value`: The value of the metadata
- `type`: The type or unit of the metadata

The transformation matrix is a 3x3 matrix used to transform coordinates from the registered measurement space to the reference measurement space.

## Get Existing Measurements

In [ ]:
# Get measurement data from the database
measurements = qrs.get_data('measurements')

# Display the first few rows
print("Available measurements:")
print(tabulate(measurements[['id_measurement', 'file_path_measurement']].head(5), 
               headers='keys', tablefmt='pretty'))

## Registration with Additional Metadata

In [ ]:
# Define file paths for another set of measurements
# Note: You should replace these with actual file paths from your measurements table
reference_file_path2 = measurements['file_path_measurement'].iloc[2]
registered_file_path2 = measurements['file_path_measurement'].iloc[3]

# Define the transformation matrix with different values
transformation_matrix2 = [
    [0.9985, -0.0550, 10.5],
    [0.0550, 0.9985, -5.2],
    [0.0, 0.0, 1.0]
]

# Define additional metadata
additional_metadata = [
    {'key': 'registration_method', 'value': 'phase correlation', 'type': 'string'},
    {'key': 'error_metric', 'value': 0.025, 'type': 'float'},
    {'key': 'iterations', 'value': 500, 'type': 'integer'},
    {'key': 'preprocessing', 'value': 'gaussian filter', 'type': 'string'}
]

# Load the registration with additional metadata
registration_id2 = load.load_registration(
    conn, 
    transformation_matrix2, 
    reference_file_path2, 
    registered_file_path2, 
    additional_metadata
)

print(f"Registration with metadata loaded with ID: {registration_id2}")

## Verify Insertion

In [ ]:
# Get registration data from the database
registration_data = qrs.get_data_metadata('registrations')

# Display the registration data
registration_data

In [ ]:
# Close the database connection
conn.close()